This notebook demonstrates how padding works for causal LLMs. It uses Llama 2.
For more details read the following article:

We only need the transformers library:

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.0 MB/s eta 0:00:00


Load the tokenizer from HF hub:

In [ ]:
from transformers import AutoTokenizer

#Replace the following with your own Hugging Face access token.
access_token = "hf_token"

#The model we want to quantize
pretrained_model_dir = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_dir, use_fast=True,  use_auth_token=access_token)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [ ]:
prompt1 = "You are not a chatbot."
prompt2 = "You are not."

In [ ]:
prompts = prompt1
input = tokenizer(prompts, return_tensors="pt");
print(input)

{'input_ids': tensor([[    1,   887,   526,   451,   263, 13563,  7451, 29889]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}


When examples has the same length, tensors can be created:

In [ ]:
prompts = [prompt1, prompt1]
input = tokenizer(prompts, return_tensors="pt");
print(input)

{'input_ids': tensor([[    1,   887,   526,   451,   263, 13563,  7451, 29889],
        [    1,   887,   526,   451,   263, 13563,  7451, 29889]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]])}


However, if they have different length, tensors can't be created. That's why we need padding.

In [ ]:
prompts = [prompt1, prompt1, prompt2]
input = tokenizer(prompts, return_tensors="pt");
print(input)

ValueError: ignored

You can use the EOS token for padding with a padding side "right". But I don't recommend it as during fine-tuning your model will that EOS token should be ignored since they are mainly used for padding (0 in the attention mask).

In [ ]:
tokenizer.padding_side = "right" #This is the default value
tokenizer.pad_token = tokenizer.eos_token
input = tokenizer(prompts, padding='max_length', max_length=20, return_tensors="pt");
print(input)

{'input_ids': tensor([[    1,   887,   526,   451,   263, 13563,  7451, 29889,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2],
        [    1,   887,   526,   451,   263, 13563,  7451, 29889,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2],
        [    1,   887,   526,   451, 29889,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [ ]:
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
input = tokenizer(prompts, padding='max_length', max_length=20, return_tensors="pt");
print(input)

{'input_ids': tensor([[    2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     1,   887,   526,   451,   263, 13563,  7451, 29889],
        [    2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     1,   887,   526,   451,   263, 13563,  7451, 29889],
        [    2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     1,   887,   526,   451, 29889]]), 'attention_mask': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]])}


Using the UNK token is much more reasonable since it rarely appears in the training examples. This strategy is much less confusing for the LLM and the padding side doesn't matter much.

In [ ]:
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.unk_token
input = tokenizer(prompts, padding='max_length', max_length=20, return_tensors="pt");
print(input)

{'input_ids': tensor([[    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     1,   887,   526,   451,   263, 13563,  7451, 29889],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     1,   887,   526,   451,   263, 13563,  7451, 29889],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     1,   887,   526,   451, 29889]]), 'attention_mask': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]])}


The best strategy is may be to create a new pad token from scratch. However, if you do that you must resize the token embeddings.

In [ ]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
input = tokenizer(prompts, padding='max_length', max_length=20, return_tensors="pt");
print(input)

{'input_ids': tensor([[32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
         32000, 32000,     1,   887,   526,   451,   263, 13563,  7451, 29889],
        [32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
         32000, 32000,     1,   887,   526,   451,   263, 13563,  7451, 29889],
        [32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
         32000, 32000, 32000, 32000, 32000,     1,   887,   526,   451, 29889]]), 'attention_mask': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]])}
